# The Command Center: WAHIS-Inspired Main Dashboard

This notebook serves as the central command center for the National Animal Health Surveillance Platform. It provides a high-level, interactive overview of the national animal health situation, inspired by professional systems like WOAH's WAHIS.

**Key Features:**
1.  **Dynamic Filters:** Slice the entire dataset by Disease, Region, and a specific Date Range.
2.  **Key Performance Indicators (KPIs):** At-a-glance summary of ongoing outbreaks, total cases, and other critical metrics.
3.  **Interactive Map:** A Folium map showing the spatial distribution of filtered outbreaks.
4.  **Charts & Graphs:** Visualizations for temporal trends and categorical breakdowns.

**Prerequisites:**
*   Ensure your `.env` file is configured with Supabase credentials.
*   Ensure all required libraries from `requirements.txt` are installed in your virtual environment.

In [ ]:
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
import matplotlib.pyplot as plt
from supabase import create_client, Client
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# --- Connect to Supabase --- 
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_ANON_KEY")

if not supabase_url or not supabase_key:
    print("❌ Supabase credentials not found. Please create a .env file.")
else:
    try:
        supabase: Client = create_client(supabase_url, supabase_key)
        print("✅ Successfully connected to Supabase.")
    except Exception as e:
        print(f"❌ Failed to connect to Supabase: {e}")

### Part 1: Data Fetching and Unification

To ensure the dashboard is fast and responsive, we perform the heavy lifting of data unification once, up front. This cell fetches all the necessary tables from the database and merges them into a single, analysis-ready master GeoDataFrame called `outbreaks_gdf`.

In [ ]:
def fetch_all_data():
    print("-> Fetching all necessary tables from Supabase...")
    try:
        # Fetch all tables in one go
        outbreaks_res = supabase.table("animal_disease_events_logbook").select("*", count='exact').execute()
        diseases_res = supabase.table("animal_diseases").select("disease_code, disease_name").execute()
        woredas_res = supabase.table("admin_woredas").select("woreda_pcode, woreda_name, zone_pcode").execute()
        zones_res = supabase.table("admin_zones").select("zone_pcode, zone_name, region_pcode").execute()
        regions_res = supabase.table("admin_regions").select("region_pcode, region_name").execute()

        df_outbreaks = pd.DataFrame(outbreaks_res.data)
        df_diseases = pd.DataFrame(diseases_res.data)
        df_woredas = pd.DataFrame(woredas_res.data)
        df_zones = pd.DataFrame(zones_res.data)
        df_regions = pd.DataFrame(regions_res.data)
        
        print(f"✅ All tables fetched successfully. Found {outbreaks_res.count} outbreak events.")
        return df_outbreaks, df_diseases, df_woredas, df_zones, df_regions

    except Exception as e:
        print(f"❌ An error occurred during data fetching: {e}")
        return (pd.DataFrame() for _ in range(5))

# Fetch and unify the data
df_outbreaks, df_diseases, df_woredas, df_zones, df_regions = fetch_all_data()

if not df_outbreaks.empty:
    print("\n-> Unifying data into a single GeoDataFrame...")
    merged_df = pd.merge(df_outbreaks, df_diseases, on='disease_code', how='left')
    merged_df = pd.merge(merged_df, df_woredas, left_on='woreda_pcode', right_on='woreda_pcode', how='left') # Explicit key
    merged_df = pd.merge(merged_df, df_zones, on='zone_pcode', how='left')
    merged_df = pd.merge(merged_df, df_regions, on='region_pcode', how='left')

    merged_df['reported_date'] = pd.to_datetime(merged_df['reported_date'])
    merged_df['latitude'] = pd.to_numeric(merged_df['latitude'], errors='coerce')
    merged_df['longitude'] = pd.to_numeric(merged_df['longitude'], errors='coerce')
    merged_df.dropna(subset=['latitude', 'longitude'], inplace=True)

    outbreaks_gdf = gpd.GeoDataFrame(
        merged_df, geometry=gpd.points_from_xy(merged_df.longitude, merged_df.latitude), crs="EPSG:4326"
    )
    print(f"✅ Master GeoDataFrame created with {len(outbreaks_gdf)} valid outbreak events.")
else:
    outbreaks_gdf = gpd.GeoDataFrame() # Create empty GDF to prevent errors
    print("⚠️ Could not create master GeoDataFrame as no outbreak data was fetched.")

### Part 2: The Interactive Dashboard

This is the main dashboard code. It defines the widgets, the layout, and the core `update_dashboard` function that filters the data and redraws all the components whenever a user changes a filter.

In [ ]:
if not outbreaks_gdf.empty:
    # --- 1. DEFINE WIDGETS (THE FILTERS) ---
    all_diseases = ['All Diseases'] + sorted(outbreaks_gdf['disease_name'].dropna().unique().tolist())
    all_regions = ['All Regions'] + sorted(outbreaks_gdf['region_name'].dropna().unique().tolist())
    
    disease_filter = widgets.Dropdown(options=all_diseases, value='All Diseases', description='Disease:')
    region_filter = widgets.Dropdown(options=all_regions, value='All Regions', description='Region:')
    start_date_filter = widgets.DatePicker(description='Start Date', value=outbreaks_gdf['reported_date'].min().date())
    end_date_filter = widgets.DatePicker(description='End Date', value=outbreaks_gdf['reported_date'].max().date())

    # --- 2. DEFINE THE DASHBOARD LAYOUT (OUTPUT AREAS) ---
    kpi_style = "font-size: 24px; font-weight: bold; color: #333; text-align: center; padding: 10px;"
    kpi_desc_style = "font-size: 12px; color: #666; text-align: center;"
    
    kpi_ongoing = widgets.HTML()
    kpi_cases = widgets.HTML()
    kpi_deaths = widgets.HTML()
    kpi_regions = widgets.HTML()

    map_output = widgets.Output()
    timeline_output = widgets.Output()
    pie_chart_output = widgets.Output()

    # --- 3. THE CORE UPDATE FUNCTION ---
    def update_dashboard(change):
        df = outbreaks_gdf.copy()
        if disease_filter.value != 'All Diseases': df = df[df['disease_name'] == disease_filter.value]
        if region_filter.value != 'All Regions': df = df[df['region_name'] == region_filter.value]
        if start_date_filter.value: df = df[df['reported_date'] >= pd.to_datetime(start_date_filter.value)]
        if end_date_filter.value: df = df[df['reported_date'] <= pd.to_datetime(end_date_filter.value)]

        # KPIs
        kpi_ongoing.value = f"<div style='{kpi_style}'>{df[df['individual_outbreak_status'] == 'Ongoing'].shape[0]:,}</div><div style='{kpi_desc_style}'>Ongoing Outbreaks</div>"
        kpi_cases.value = f"<div style='{kpi_style}'>{df['cases'].sum():,}</div><div style='{kpi_desc_style}'>Total Cases</div>"
        kpi_deaths.value = f"<div style='{kpi_style}'>{df['deaths'].sum():,}</div><div style='{kpi_desc_style}'>Total Deaths</div>"
        kpi_regions.value = f"<div style='{kpi_style}'>{df['region_name'].nunique():,}</div><div style='{kpi_desc_style}'>Affected Regions</div>"

        # Map
        with map_output: 
            clear_output(wait=True)
            m = folium.Map(location=[9,40], zoom_start=6, tiles='CartoDB dark_matter')
            if not df.empty: [folium.CircleMarker([r.geometry.y, r.geometry.x], radius=5, color='orange', fill=True, popup=r['disease_name']).add_to(m) for _, r in df.iterrows()]
            display(m)

        # Timeline Chart
        with timeline_output: 
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(6,3));
            if not df.empty: df.set_index('reported_date').resample('M').size().plot(ax=ax, marker='o');
            ax.set_title('Outbreaks Over Time (Monthly)'); ax.grid(True, linestyle='--'); plt.tight_layout(); plt.show()

        # Pie Chart
        with pie_chart_output:
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(6,3));
            if not df.empty: df['disease_name'].value_counts().nlargest(5).plot(kind='pie', ax=ax, autopct='%1.1f%%');
            ax.set_title('Top 5 Diseases'); ax.set_ylabel(''); plt.tight_layout(); plt.show()
            
    # --- 4. LINK WIDGETS AND DISPLAY ---
    for w in [disease_filter, region_filter, start_date_filter, end_date_filter]: w.observe(update_dashboard, names='value')

    filters_box = widgets.HBox([disease_filter, region_filter])
    date_filters_box = widgets.HBox([start_date_filter, end_date_filter])
    kpi_box = widgets.HBox([kpi_ongoing, kpi_cases, kpi_deaths, kpi_regions], layout=widgets.Layout(justify_content='space-around'))
    charts_box = widgets.VBox([timeline_output, pie_chart_output])
    main_content_box = widgets.HBox([map_output, charts_box], layout=widgets.Layout(align_items='stretch'))
    
    dashboard = widgets.VBox([
        widgets.HTML("<h1>National Animal Health Surveillance Dashboard</h1>"),
        filters_box, date_filters_box, widgets.HTML("<hr>"), kpi_box, widgets.HTML("<hr>"), main_content_box
    ])
    
    print("✅ Dashboard is ready.")
    display(dashboard)
    update_dashboard(None) # Initial run
else:
    print("⚠️ Dashboard cannot be displayed because no data was loaded.")